In [19]:
import os
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import time
import math
import json
import random

In [5]:
%matplotlib inline

In [45]:
with tf.Graph().as_default(): 
    images, labels, keypts = distorted_inputs(10)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    print sess.run([labels, images])

[array([ 3, 29, 28,  3, 36, 28, 13,  9, 32, 11], dtype=int32), array([[[[ 0.6016826 ,  0.24945885, -0.42792562],
         [ 0.61699253,  0.26476875, -0.41261572],
         [ 0.62177151,  0.26954752, -0.40783694],
         ..., 
         [ 0.86283624,  0.57087857, -0.14668339],
         [ 0.86283624,  0.57087857, -0.14668339],
         [ 0.86283624,  0.57087857, -0.14668339]],

        [[ 0.62177151,  0.26954752, -0.40783694],
         [ 0.62177151,  0.26954752, -0.40783694],
         [ 0.62177151,  0.26954752, -0.40783694],
         ..., 
         [ 0.8580575 ,  0.56609976, -0.15146215],
         [ 0.84274757,  0.55078965, -0.16677207],
         [ 0.84274757,  0.55078965, -0.16677207]],

        [[ 0.62177151,  0.26954752, -0.40783694],
         [ 0.62177151,  0.26954752, -0.40783694],
         [ 0.62450337,  0.27227965, -0.40510482],
         ..., 
         [ 0.85482025,  0.56286252, -0.15469943],
         [ 0.84274757,  0.55078965, -0.16677207],
         [ 0.84274757,  0.55078965, -0

In [44]:
def distorted_inputs(batch_size):
    img_list = []
    label_list = []
    keypts_list = []
    i = 0
    for folder in os.listdir('train'):
        if folder == '.DS_Store':
            continue
        cur_list = ['train/' + folder + '/' + img 
                    for img in os.listdir('train/' + folder) if img.endswith('.jpg')]
        label_list.extend([i] * len(cur_list))
        keypts_list.extend(get_keypts(folder, cur_list, train=True))
        img_list.extend(cur_list)
        i += 1
    
    input_tensor = zip(img_list, label_list, keypts_list)
    random.shuffle(input_tensor)
    
    data_queue = tf.FIFOQueue(capacity=100, dtypes=[tf.string, tf.int32, tf.float32], shapes=[[],[],[36]])
    enqueue_op = data_queue.enqueue_many(zip(*input_tensor))
    qr = tf.train.QueueRunner(data_queue, [enqueue_op] * 4)
    tf.train.add_queue_runner(qr)
    
    img_file, label, keypt = data_queue.dequeue()
    raw = tf.read_file(img_file)
    img = tf.image.decode_jpeg(raw)
    resized_img = tf.image.resize_images(img, tf.constant([227, 227]))
    fliped_img = tf.image.random_flip_left_right(resized_img)
    distorted_img = tf.image.random_brightness(fliped_img, max_delta=50)
    distorted_img = tf.image.random_contrast(distorted_img, lower=0.2, upper=1.8)
    float_img = tf.image.per_image_standardization(distorted_img)
    float_img.set_shape([227, 227, 3])
    images, labels, keypts = tf.train.shuffle_batch([float_img, label, keypt],
                                   batch_size=batch_size,
                                   capacity=1000 + 3 * batch_size,
                                   min_after_dequeue=1000)
    return images, labels, keypts

In [66]:
def inputs(batch_size):
    img_list = []
    label_list = []
    keypts_list = []
    i = 0
    for folder in os.listdir('test'):
        if folder == '.DS_Store':
            continue
        cur_list = ['test/' + folder + '/' + img 
                    for img in os.listdir('test/' + folder) if img.endswith('.jpg')]
        label_list.extend([i] * len(cur_list))
        keypts_list.extend(get_keypts(folder, cur_list, train=False))
        img_list.extend(cur_list)
        i += 1
        
    data_queue = tf.FIFOQueue(capacity=50, dtypes=[tf.string, tf.int32, tf.float32], shapes=[[],[],[36]])
    enqueue_op = data_queue.enqueue_many([img_list, label_list, keypts_list])
    qr = tf.train.QueueRunner(data_queue, [enqueue_op] * 4)
    tf.train.add_queue_runner(qr)
    
    img_file, label, keypt = data_queue.dequeue()
    raw = tf.read_file(img_file)
    img = tf.image.decode_jpeg(raw)
    
    resized_img = tf.image.resize_images(img, tf.constant([227, 227]))
    float_img = tf.image.per_image_standardization(resized_img)
    float_img.set_shape([227, 227, 3])
    images, labels, keypts  = tf.train.batch([float_img, label, keypt],
                                    batch_size=batch_size,
                                    capacity=100 + batch_size)
    return images, labels, keypts

In [2]:
def extract_train_test():
    train_map = {}
    for annotations in os.listdir('ImageSplits'):
        if annotations == 'actions.txt':
            continue
        if annotations.endswith('train.txt'):
            cls = '_'.join(annotations.split('_')[:-1])
            if cls not in train_map:
                train_map[cls] = set()
            with open('ImageSplits/' + annotations) as f:
                for line in f:
                    train_map[cls].add(line.strip())
            train_folder = 'train/' + cls
            test_folder = 'test/' + cls
            if not os.path.exists(train_folder):
                os.makedirs(train_folder)
            if not os.path.exists(test_folder):
                os.makedirs(test_folder)
    for img_file in os.listdir('JPEGImages'):
        cls = '_'.join(img_file.split('_')[:-1])
        if img_file in train_map[cls]:
            os.rename('JPEGImages/' + img_file, 'train/' + cls + '/' + img_file)
        else:
            os.rename('JPEGImages/' + img_file, 'test/' + cls + '/' + img_file)

In [3]:
def rescale_keypts(train):
    if train:
        path = 'train/'
        keypts_path = 'keypts_train/'
    else:
        path = 'test/'
        keypts_path = 'keypts_test/'
    for folder in os.listdir('keypts'):
        if folder == '.DS_Store':
            continue
        for keypts_file in os.listdir(keypts_path + folder):
            if not keypts_file.endswith('.json'):
                continue
            with open(keypts_path + folder + '/' + keypts_file) as f:
                keypts_js = json.load(f)
            if len(keypts_js['people']) != 1:
                continue
            img_file = path + folder + '/' + '_'.join(keypts_file.split('_')[:-1]) + '.jpg'
            base = keypts_file.rstrip('.json')
            orig_shape = plt.imread(img_file).shape
            ratio = (227.0 / orig_shape[1], 227.0 / orig_shape[0])
            if not os.path.exists(path + folder + '/scaled_keypts'):
                os.mkdir(path + folder + '/scaled_keypts')
            with open(path + folder + '/scaled_keypts/' + base + '_scaled.json', 'wb+') as f:
                people = keypts_js['people'][0]
                keypts = people['pose_keypoints']
                x = keypts[::3]
                y = keypts[1::3]
                c = keypts[2::3]
                
                xs = list(np.array(x) * ratio[0])
                ys = list(np.array(y) * ratio[1])
                
                json.dump(zip(xs, ys), f)
        print '%s done'%folder

In [4]:
def get_keypts(folder, img_list, train):
    results = []
    i = 0
    j = 0
    if train:
        keypts_files = os.listdir('train/' + folder + '/scaled_keypts')
    else:
        keypts_files = os.listdir('test/' + folder + '/scaled_keypts')
    while i < len(img_list) and j < len(keypts_files):
        img_num = img_list[i].rstrip('.jpg').split('_')[-1]
        result = []
        if img_num in keypts_files[j]:
            if train:
                with open('train/' + folder + '/scaled_keypts/' + keypts_files[j]) as js_f:
                    js = json.load(js_f)
            else:
                with open('test/' + folder + '/scaled_keypts/' + keypts_files[j]) as js_f:
                    js = json.load(js_f)
            result = [val for pair in js for val in pair]
            i += 1
            j += 1
        else:
            result = [0.0] * (18*2)
            i += 1
        results.append(result)
    while i < len(img_list):
        results.append([0.0] * 36)
        i += 1
    return results

In [7]:
def add_summary(x):
    tf.summary.scalar(x.op.name + '/min', tf.reduce_min(x))
    tf.summary.scalar(x.op.name + '/max', tf.reduce_max(x))
    tf.summary.scalar(x.op.name + '/mean', tf.reduce_mean(x))

In [8]:
def inference(images, keep_prob):
     # 1st layer
    with tf.variable_scope('conv1') as scope:
        kernel = tf.get_variable('weights', [11, 11, 3, 64], tf.float32, 
                                 tf.truncated_normal_initializer(stddev=1e-1))
        kernel = tf.verify_tensor_all_finite(kernel, 'kernel1 error')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
        biases = tf.get_variable('biases', [64], tf.float32, tf.constant_initializer(0.0))
        conv1 = tf.nn.relu(conv + biases, name=scope.name)
        conv1 = tf.verify_tensor_all_finite(conv1, 'conv1 error: ')
        
    lrn1 = tf.nn.local_response_normalization(conv1, alpha=1e-4, beta=0.75,
                                              depth_radius=2, bias=2.0, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    
    # 2nd layer
    with tf.variable_scope('conv2') as scope:
        kernel = tf.get_variable('weights', [5, 5, 64, 192], tf.float32, 
                                 tf.truncated_normal_initializer(stddev=1e-1))
        kernel = tf.verify_tensor_all_finite(kernel, 'kernel2 error')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.get_variable('biases', [192], tf.float32, tf.constant_initializer(0.0))
        conv2 = tf.nn.relu(conv + biases, name=scope.name)
        conv2 = tf.verify_tensor_all_finite(conv2, 'conv2 error: ')
    
    lrn2 = tf.nn.local_response_normalization(conv2, alpha=1e-4, beta=0.75, depth_radius=2, bias=2.0, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
        
    
    with tf.variable_scope('fc1') as scope:
        W_fc1 = tf.get_variable('weights', [13 * 13 * 192, 1024], tf.float32, 
                                tf.truncated_normal_initializer(stddev=1e-1))
        b_fc1 = tf.get_variable('biases', [1024], tf.float32, tf.constant_initializer(0.0))
        conv2_flat = tf.reshape(pool2, [-1, 13 * 13 * 192])
        fc1 = tf.nn.relu(tf.matmul(conv2_flat, W_fc1) + b_fc1, name=scope.name)
        fc1 = tf.verify_tensor_all_finite(fc1, 'fc1 error: ')
    
    # drop out
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
    # read out layer
    with tf.variable_scope('fc2') as scope:
        W_fc2 = tf.get_variable('weights', [1024, 40], tf.float32, tf.truncated_normal_initializer(stddev=1e-1))
        b_fc2 = tf.get_variable('biases', [40], tf.float32, tf.constant_initializer(0.0))
        y_conv = tf.add(tf.matmul(fc1_drop, W_fc2), b_fc2, name=scope.name)
        y_conv = tf.verify_tensor_all_finite(y_conv, 'y_conv error: ')
    
    return y_conv

In [52]:
def conv_relu(name, inputs, k_size, stride, padding, groups):
    # k_size needs to be [h, w, num_in, num_out]
    input_channels = int(inputs.get_shape()[-1])
    assert input_channels % groups == 0
    assert k_size[3] % groups == 0
    with tf.variable_scope(name) as scope:
        k_size[2] /= groups
        kernel = tf.get_variable('weights', k_size, tf.float32, 
                                 tf.truncated_normal_initializer(stddev=math.sqrt(2.0/(np.prod(k_size[:3])))),
                                 tf.nn.l2_loss)
        biases = tf.get_variable('biases', [k_size[3]], tf.float32, tf.constant_initializer(0.0))
        if groups == 1:
            conv = tf.nn.conv2d(inputs, kernel, stride, padding=padding)
        else:
            input_groups = tf.split(inputs, groups, 3)
            kernel_groups = tf.split(kernel, groups, 3)
            output_groups = [tf.nn.conv2d(i, k, stride, padding=padding) for i, k in zip(input_groups, kernel_groups)]
            conv = tf.concat(output_groups, 3)
            
        conv = tf.verify_tensor_all_finite(conv, name + ' infinite error!!!')
        return tf.nn.relu(conv + biases, name=scope.name)
        

In [53]:
def fc(name, inputs, output_size, relu=True):
    # inputs should be flattened to a 2D tensor
    input_size = int(inputs.get_shape()[1])
    with tf.variable_scope(name) as scope:
        W = tf.get_variable('weights', [input_size, output_size], tf.float32, 
                            tf.truncated_normal_initializer(stddev=math.sqrt(2.0/input_size)),
                            tf.nn.l2_loss)
        b = tf.get_variable('biases', [output_size], tf.float32, tf.constant_initializer(0.0))
        out = tf.verify_tensor_all_finite(tf.matmul(inputs, W) + b, name + ' inifite error!!!')
        if relu:
            return tf.nn.relu(out, name=scope.name)
        else:
            return out

In [46]:
def inference_deep(images, keep_prob, keypts=None):
    # 1st layer
    conv1 = conv_relu('conv1', images, [11, 11, 3, 96], [1, 4, 4, 1], 'VALID', 1)
    lrn1 = tf.nn.local_response_normalization(conv1, alpha=2e-5, beta=0.75,
                                              depth_radius=2, bias=1.0, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], 
                           strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    
    # 2nd layer
    conv2 = conv_relu('conv2', pool1, [5, 5, 96, 256], [1, 1, 1, 1], 'SAME', 2)
    lrn2 = tf.nn.local_response_normalization(conv2, alpha=2e-5, beta=0.75, 
                                              depth_radius=2, bias=1.0, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], 
                           strides=[1, 2, 2, 1], padding='VALID', name='pool2')
    
    # 3rd layer
    conv3 = conv_relu('conv3', pool2, [3, 3, 256, 384], [1, 1, 1, 1], 'SAME', 1)
        
    # 4th layer
    conv4 = conv_relu('conv4', conv3, [3, 3, 384, 384], [1, 1, 1, 1], 'SAME', 2)
        
    # 5th layer
    conv5 = conv_relu('conv5', conv4, [3, 3, 384, 256], [1, 1, 1, 1], 'SAME', 2)
    pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool5')
    
    # 6th layer
    pool5_flat = tf.reshape(pool5, [-1, 6 * 6 * 256])
    if keypts is not None:
        pool5_flat = tf.concat([pool5_flat, keypts], axis=1)
    
    fc6 = fc('fc6', pool5_flat, 4096)
    
    # drop out
    fc6_drop = tf.nn.dropout(fc6, keep_prob)
    
    # 7th layer
    fc7 = fc('fc7', fc6_drop, 4096)
    
    # drop out
    fc7_drop = tf.nn.dropout(fc7, keep_prob)
    
    # readout layer
    fc8 = fc('fc8', fc7_drop, 40, relu=False)
    
    return fc8

In [47]:
def load_pretrained_weights(skip_layers, set_untrainable=True, warm_start=False):
    weights = np.load('bvlc_alexnet.npy').item()
    ops = []
    trainables = tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES)
    regularizations = tf.get_collection_ref(tf.GraphKeys.REGULARIZATION_LOSSES)
    for layer in weights:
        if layer not in skip_layers or warm_start:
            with tf.variable_scope(layer, reuse=True) as scope:
                kernel = tf.get_variable('weights')
                ops.append(tf.assign(kernel, weights[layer][0]))
                biases = tf.get_variable('biases')
                ops.append(tf.assign(biases, weights[layer][1]))
                if set_untrainable:
                    trainables.remove(kernel)
                    trainables.remove(biases)
                    regularizations.remove(tf.losses.get_regularization_losses(scope.name)[0])
    return tf.group(*ops)

In [48]:
def loss(logits, labels, wd):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=labels, logits=logits))
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    return cross_entropy + wd * tf.add_n(reg_losses)

In [49]:
def train(loss, global_step):
    #lr = tf.train.exponential_decay(0.01, global_step, 3000, 0.1, True)
    #opt = tf.train.GradientDescentOptimizer(lr)
    opt = tf.train.AdamOptimizer()
    train_op = opt.minimize(loss, global_step)
#     variable_averages = tf.train.ExponentialMovingAverage(0.999, global_step)
#     variable_averages_op = variable_averages.apply(tf.trainable_variables())
#     with tf.control_dependencies([train_op, variable_averages_op]):
#         op = tf.no_op()
#     return op
    return train_op

In [50]:
def main(train_from_scratch=False):
    with tf.Graph().as_default() as g:
        global_step = tf.contrib.framework.get_or_create_global_step()
        with tf.device('/cpu:0'):
            images, labels, keypts = distorted_inputs(128)

        pred = inference_deep(images, 0.5)
        skip_layers = ['fc7', 'fc8']
        load_op = load_pretrained_weights(skip_layers)
        total_loss = loss(pred, labels, 0.0)
        train_op = train(total_loss, global_step)

        class _LoggerHook(tf.train.SessionRunHook):
            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                if self._step % 100 == 0:
                    return tf.train.SessionRunArgs(total_loss)
                else:
                    return None

            def after_run(self, run_context, run_values):
                if self._step % 100 == 0 :
                    current_time = time.time()
                    self._start_time = current_time
                    loss_val = run_values.results
                    format_str = '%s: step %d, loss = %.3f'
                    print format_str % (datetime.now(), self._step, loss_val)
                if self._step > 0 and self._step % 500 == 0:
                    evaluate()

        with tf.train.MonitoredTrainingSession(
            checkpoint_dir='./tmp/ckpt_2',
            hooks=[tf.train.StopAtStepHook(last_step=10000),
                  tf.train.NanTensorHook(total_loss),
                  _LoggerHook()]
        ) as sess:
            if not train_from_scratch:
                sess.run(load_op)
            while not sess.should_stop():
                sess.run(train_op)

In [54]:
main()

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/ckpt_2/model.ckpt-485
INFO:tensorflow:Saving checkpoints for 485 into ./tmp/ckpt_2/model.ckpt.
2017-09-11 21:35:41.888985: step 0, loss = 3.617
INFO:tensorflow:global_step/sec: 0.270912
2017-09-11 21:41:48.674516: step 100, loss = 2.739
INFO:tensorflow:Saving checkpoints for 646 into ./tmp/ckpt_2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.240859
2017-09-11 21:48:43.859718: step 200, loss = 2.717


KeyboardInterrupt: 

In [105]:
def evaluate(train=False, batch_size=1000):
    with tf.Graph().as_default() as g:
        if train:
            images, labels, _ = distorted_inputs(batch_size)
        else:
            images, labels, _ = inputs(batch_size)
        pred = inference_deep(images, 1.0)
        top_k_op = tf.nn.in_top_k(pred, labels, 1)
        variable_averages = tf.train.ExponentialMovingAverage(0.0)
        variables_to_restore = variable_averages.variables_to_restore()
#         fix_v = {v.op.name: v for v in tf.trainable_variables() if not v.op.name.startswith('fc7') and not v.op.name.startswith('fc8')}
#         ema_v = {k: v for k, v in variables_to_restore.items() if k.startswith('fc7') or k.startswith('fc8')}
#         fix_v.update(ema_v)
#         print fix_v
        saver = tf.train.Saver()
#         summary_op = tf.summary.merge_all()
#         summary_write = tf.summary.FileWriter('./tmp/eval', g)
        with tf.Session() as sess:
            ckpt = tf.train.get_checkpoint_state('./tmp/ckpt_2')
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.all_model_checkpoint_paths[-2])
            else:
                print 'No checkpoint file found'
                return
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess, coord)
            try:
                true_count = np.sum(sess.run(top_k_op))
                precision = float(true_count) / batch_size
                print 'precision: %.3f' % precision
            except Exception as e:
                print e
            coord.request_stop()
            coord.join(threads, stop_grace_period_secs=10)
            

In [109]:
evaluate(False, 1000)

INFO:tensorflow:Restoring parameters from ./tmp/ckpt_2/model.ckpt-323
precision: 0.105


In [26]:
N_SAMPLES = 20
data = 10 * np.random.randn(N_SAMPLES, 4) + 1
target = np.random.randint(0, 2, size=N_SAMPLES

In [50]:
NUM_THREADS = 4
queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])
# queue  = tf.train.input_producer([data, target])
enqueue_op = queue.enqueue_many([data, target])
data_batch, label_batch = queue.dequeue()
# create NUM_THREADS to do enqueue
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)
with tf.Session() as sess:
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in xrange(100): # do to 100 iterations
        if coord.should_stop():
            break
        print 'label: ', sess.run(data_batch)
    coord.request_stop()
    coord.join(enqueue_threads)

label:  [-15.76484776  -1.97194302 -12.6300869    6.41410542]
label:  [ 11.14868164   3.56267858 -15.04469109  -9.80371094]
label:  [  8.03130531  10.71555138  -8.06960106 -20.6381073 ]
label:  [-3.04018426  0.78890592 -5.47052765  0.58360696]
label:  [-14.27897549   6.81375933  -6.26739216  -2.98046756]
label:  [ 7.49180937  5.41993713 -1.01845872  7.99767256]
label:  [  7.01992083  20.8553791   -0.63825244  -9.48780346]
label:  [ -3.0049963  -11.29572868   0.87475055  -8.48777485]
label:  [ 34.11064148  22.09499931   8.56626797  -1.86508656]
label:  [ 13.01100731 -11.10157585   2.16470504 -11.60906315]
label:  [ -9.11300278  21.20376587  -1.62195039   9.54562569]
label:  [-19.14703941   1.41451061  -5.62721634  -2.7678473 ]
label:  [ -1.06307101  -6.5821166  -15.47267818  -6.13916779]
label:  [-3.30766511  3.53310657  0.40748918 -6.85776377]
label:  [ 2.07924652 -0.27916488 -7.1269474   9.59386635]
label:  [ 15.72919464   6.80227566   2.41824603   0.81573993]
label:  [ -7.09694862  -